In [1]:
import glob 
import numpy as np 
import matplotlib.pyplot as plt 

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img 

%matplotlib inline

2023-01-27 15:02:05.283318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'load_img' from 'keras.preprocessing.image' (/Users/michielbontenbal/.local/lib/python3.9/site-packages/keras/preprocessing/image.py)

### De cel hieronder is de functie om de gezichten uit de fotos te knippen.

In [ ]:
import glob 
import cv2
import sys
import uuid

def image_get_faces(image_name, location):
    image = cv2.imread(image_name)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(100, 100)
    )

    for (x, y, w, h) in faces:
        print("[INFO] Found {0} Faces.".format(len(faces)), end=' ')
        print((x, y, w, h), end=' ') 
        print("Object found, saved.")
        tempname = str(uuid.uuid4().hex)
        clip = cv2.resize(image[y:y + h, x:x + w],(150,150))
        cv2.imwrite(str(location) + tempname + '_faces.jpg', clip)

In [ ]:
IMG_DIM = (150, 150)

# De labels worden uit het aantal verschillende filenamen gehaald.(de nummers en haakjes worden gewist)

train_files = glob.glob('faces2\\*.jpg')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('\\')[1].split('.')[0].rstrip(' (12345678900)') for fn in train_files]

unknown_files = glob.glob('faces-test\\*.jpg')
unknown_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in unknown_files]
unknown_imgs = np.array(unknown_imgs)
unknown_labels = [fn.split('\\')[1].split('.')[0].rstrip(' (12345678900)') for fn in unknown_files]


### Hieronder begint de data augmentation het genereren van extra images.

In [ ]:
train_datagen =  ImageDataGenerator(rescale=1./255, zoom_range=0.2, rotation_range=20,
                                    width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, 
                                    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

unknown_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# zet de namen om in de zogenaamde onehot encoding.

encoder = LabelBinarizer()
transformed_label = encoder.fit_transform(train_labels)
unknown_transformed_label = encoder.transform(unknown_labels)

train_generator = train_datagen.flow(train_imgs, transformed_label, batch_size=64)
val_generator = val_datagen.flow(train_imgs, transformed_label,batch_size=64)
unknown_generator = val_datagen.flow(unknown_imgs, unknown_transformed_label,batch_size=64)

# letop als je een ander aantal personen hebt moet je dit aanpassen.
def test_generator(generator, nr = 10 ):
    names = [ 'Persoon 1' , 'Persoon 2'  , 'Persoon 3', 'Persoon 3']
    fig, ax = plt.subplots(1,nr, figsize=(nr, 20), frameon=False)
    vals, labs = next(generator)
    for i in range(0,nr):
        ax[i].imshow(vals[i])
        ax[i].set_frame_on(False)
        ax[i].set_title(names[np.argmax(labs[i])])
        
test_generator(unknown_generator, nr=32)


In [ ]:
from keras.applications import vgg16
from keras.models import Model
import keras
import pandas as pd

# Kies hieronder ook eens ander net, of wat gebeurt er als je een getraind net neemt
# Neem een geinitialiseert netwerk met imagenet weights 
# Werkt een ander netwerk beter b.v. Resnet e.e.a.

input_shape = (150, 150, 3)
vgg = vgg16.VGG16(include_top=False, weights=None, input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)


# Hieronder stel je in hoeveel lagen van het VGG netwerk je wil bij trainen. Nu werkt het niet
# het netwerk heeft random weights en kan niet getraiond worden.

vgg_model = Model(vgg.input, output)
vgg_model.trainable = True
set_trainable = True
for layer in vgg_model.layers:
    if layer.name in ['block5_conv_NOOOIIIT']:  # vanaf de naam van de layer tot aan outputlaag is te trainen.
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

vgg_model.summary() 

# Print layer status.
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU, Dropout
from keras import optimizers
import shutil as sh
import tensorflow as tf

tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=0, write_graph=True, write_images=True)

sh.rmtree('logs',ignore_errors=True)
callbacks = [tbCallBack]

model = Sequential()
model.add(vgg_model) # Stop het complete vgg model in het nieuwe model.

model.add(Dense(4, activation='softmax'))  # voeg je eigen lagen toe, of is dit genoeg?

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

In [ ]:
callbacks = [tbCallBack]

sh.rmtree('logs',ignore_errors=True)
history = model.fit(train_generator,  steps_per_epoch=2, epochs=50,
                              validation_data=unknown_generator, validation_steps=50,
                              verbose=1,callbacks=callbacks)


In [ ]:
x , y = next(unknown_generator)

results = model.predict(x)

names = [ 'Persoon 1' , 'Persoon 2' , 'Persoon 3', 'Persoon 4']
tl = [ names[np.argmax(res)] for res in results] 


In [ ]:
fig, ax = plt.subplots(1,10, figsize=(30, 12))
for i in range(0,10):
    ax[i].imshow(x[i])
    ax[i].set_title(tl[i])

fig, ax = plt.subplots(1,10, figsize=(30, 12))
for i in range(10,20):
    ax[i-10].imshow(x[i])
    ax[i-10].set_title(tl[i])

fig, ax = plt.subplots(1,10, figsize=(30, 12))
for i in range(20,30):
    ax[i-20].imshow(x[i])
    ax[i-20].set_title(tl[i])



### De code hieronder laat zien hoe je een model kan opslaan en later weer gebruiken.

In [ ]:
model.save('Frozen-cnn-100ep-100bat-size100-perfect-2.h5')

In [ ]:
from keras.models import load_model

model = load_model('Frozen-cnn-100ep-100bat-size100-perfect-2.h5')

In [ ]:
x , y = next(unknown_generator)
test_loss, test_acc = model.evaluate(x, y)

tl = [ names[np.argmax(res)] for res in y] 
fig, ax = plt.subplots(1,31, figsize=(30, 12))
for i in range(0,31):
    ax[i].imshow(x[i])
    ax[i].set_title(tl[i])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'loss'], loc='upper left')
plt.show()